In [51]:
print('hello world')

hello world


In [52]:
# imports 
import torch 
import math
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from datasets import load_from_disk
from transformers import AutoTokenizer
from Training_utils.buildModel import BuildModel
from Training_utils.training_tools import Train_tools
from functools import partial


Data and Tokenizer Loading 

In [53]:
# loading tokenizer 

tokenizer_path = '../Full Pipeline(LLM)/Saved_tokenizer/t5_Tokinzer'
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path,use_fast=False)

In [54]:

# preproccesd data load
path = '/home/gz/Documents/Full Pipeline(LLM)/Saved_Data/processedDataset'
dataset = load_from_disk(path)

In [55]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')

helper Variables info

In [56]:
vocab_size = len(tokenizer)
ignore_index = tokenizer.pad_token_type_id
device = ('cuda' if torch.cuda.is_available() else 'cpu')
max_len = 80

DataLoader and Data split


In [57]:
dataset

Dataset({
    features: ['input_ids', 'text_sample'],
    num_rows: 104161
})

In [58]:
for sample in dataset:
    print(sample['input_ids'])
    print(sample['text_sample'])
    break

[32101, 32103, 497, 3, 6, 3, 354, 603, 3, 6, 149, 81, 352, 21, 3, 9, 360, 36, 277, 227, 2634, 3, 58, 32100, 32104, 25, 214, 24, 19, 24873, 68, 19, 310, 59, 207, 21, 69, 4639, 3, 5, 32100]
<start> <user> say , jim , how about going for a few beers after dinner ? <end> <bot> you know that is tempting but is really not good for our fitness . <end>


In [59]:
#collate fn 

def collate_fn(batch,pad_token=ignore_index,device=device,max_length=max_len):
   
    
   

    inputs_list = []
    target_list = []
    truncated_seqs = []
    for item in batch:
        inputIds = item['input_ids']
        if max_length is not None and len(inputIds) > max_length:
            inputIds = inputIds[:max_length-1] + [inputIds[-1]]
        truncated_seqs.append(inputIds)

    batch_max_len = max(len(seq) + 1 for seq in truncated_seqs)
    
    for inputIds in truncated_seqs:

        new_input = list(inputIds) + [pad_token]
       
        padded = (
            new_input + [pad_token] * (batch_max_len - len(new_input)))
    
        inputs_list.append(torch.tensor(padded[:-1],dtype=torch.long,device=device))
        target_list.append(torch.tensor(padded[1:],dtype=torch.long,device=device))

        
    input_ids = torch.stack(inputs_list)
    labels = torch.stack(target_list)


    return (input_ids,
            labels)
   

collate_fn_pre_loaded = partial(collate_fn,pad_token=ignore_index,device=device,max_length=285)

In [60]:
from functools import partial

collate_fn_pre_loaded = partial(collate_fn,pad_token=ignore_index,device=device,max_length=80)

In [61]:
# removing text column 

dataset = dataset.remove_columns(['text_sample'])
dataset = dataset.shuffle(seed=42)

In [62]:
train_percentage = int(len(dataset) * 0.95)
test_percentage = int(len(dataset)* 0.001)
val_percentage = len(dataset) - train_percentage - test_percentage

print(f'train Portion: {train_percentage}')
print(f'val portion: {val_percentage}')
print(f'test portion: {test_percentage}')


train Portion: 98952
val portion: 5105
test portion: 104


In [63]:
# datset split
train_data = dataset.select(range(train_percentage))
val_data = dataset.select(range(train_percentage, train_percentage+val_percentage))
test_data = dataset.select(range(train_percentage+val_percentage,len(dataset)))
print(f'Train Samples: {len(train_data)}')
print(f'Validation Samples: {len(val_data)}')
print(f'Test Samples: {len(test_data)}')


Train Samples: 98952
Validation Samples: 5105
Test Samples: 104


In [64]:
# DataLoaders
Train_loader = DataLoader(train_data,batch_size=4,collate_fn=collate_fn_pre_loaded,shuffle=True,drop_last=False)
Val_Loader = DataLoader(val_data,batch_size=4,collate_fn=collate_fn_pre_loaded,shuffle=False,drop_last=False)
Test_loader = DataLoader(test_data,batch_size=4,shuffle=False,drop_last=False)


In [65]:
num_batches = len(Train_loader)

Model and Training Config

In [66]:
model_config = {
    'D_Model': 420,
    'Num_Heads': 6,
    'Num_Layers':6,
    'Dropout': 0.05,    
    'Vocab_size': vocab_size,
    'FeedForward_size': 2000,
    'Context_size':80

}


train_config = {
        'learning_rate':0.0001, 

        'GradAccumulation_steps': 1,
        'Weight_decay': 0.01,
        'Epochs': 2,
        'Label_smoothing': 0.00,
        'warmupsteps_percentage':0.30,
        'Num_batches': num_batches,
        'ignore_index': ignore_index,
    

}



In [67]:
# create model
builder = BuildModel(model_type='GModel2')
model = builder.createModel(config=model_config, Model_type='gmodel2')
model.to(device)


state = torch.load(
    '/home/gz/Documents/Full Pipeline(LLM)/Saved_Models/45m_285T_Pretrained/45m285T_pretrained.pth',
    map_location=device
)

# change pe for finetuning 
max_len = model_config["Context_size"]  
state["positional_encoding.pe"] = state["positional_encoding.pe"][:, :max_len, :]


model.load_state_dict(state, strict=False)



/tmp/ipykernel_7277/3188294650.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(


<All keys matched successfully>

In [68]:
print(builder.get_total_params(in_millons=True))

41.343965


In [41]:
for name,param in model.named_parameters():
    print(name)

token_embedding.weight
transformer_blocks.0.Attention.WQ.weight
transformer_blocks.0.Attention.WQ.bias
transformer_blocks.0.Attention.WK.weight
transformer_blocks.0.Attention.WK.bias
transformer_blocks.0.Attention.WV.weight
transformer_blocks.0.Attention.WV.bias
transformer_blocks.0.Attention.Final_layer.weight
transformer_blocks.0.Attention.Final_layer.bias
transformer_blocks.0.FeedForward.fc1.weight
transformer_blocks.0.FeedForward.fc1.bias
transformer_blocks.0.FeedForward.fc2.weight
transformer_blocks.0.FeedForward.fc2.bias
transformer_blocks.0.RMSNorm1.scale
transformer_blocks.0.RMSNorm2.scale
transformer_blocks.1.Attention.WQ.weight
transformer_blocks.1.Attention.WQ.bias
transformer_blocks.1.Attention.WK.weight
transformer_blocks.1.Attention.WK.bias
transformer_blocks.1.Attention.WV.weight
transformer_blocks.1.Attention.WV.bias
transformer_blocks.1.Attention.Final_layer.weight
transformer_blocks.1.Attention.Final_layer.bias
transformer_blocks.1.FeedForward.fc1.weight
transformer_b

In [69]:
for name, param in model.named_parameters():
    if name.startswith("token_embedding"):
        param.requires_grad = False
    elif any(name.startswith(f"transformer_blocks.{i}.") for i in range(3)):
        param.requires_grad = False
    else:
        param.requires_grad = True


In [114]:
for name, param in model.named_parameters():
  
    if any(name.startswith(f"transformer_blocks.{i}.") for i in range(3)):
        param.requires_grad = False
    else:
        param.requires_grad = True  


In [70]:
# Training tools

loss_fn,optimzer,lr_scheduler = Train_tools(config=train_config).getTools(model.parameters())

In [71]:
def train(model, optimizer, lr_scheduler, loss_fn, Train_Loader, Val_Loader, Epochs, grad_step, pad_token, device):
    #scaler = GradScaler()
    for epoch in range(Epochs):
        progress_bar = tqdm(Train_Loader, desc=f"Epoch {epoch+1} training", leave=False)
        total_loss = 0.0
        model.train()
        for step, (input_ids, labels) in enumerate(progress_bar):
            with torch.autocast(device_type=device, dtype=torch.bfloat16): 
                logits = model(input_ids, pad_token)
                loss = loss_fn(logits.view(-1, logits.size(-1)), labels.view(-1))
            total_loss += loss.item()
            loss = loss / grad_step
            loss.backward()  
            if (step + 1) % grad_step == 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
                lr_scheduler.step()
                '''min_lr = 5e-5
                for g in optimizer.param_groups:
                    g['lr'] = max(g['lr'], min_lr)'''
                optimizer.zero_grad()
            progress_bar.set_postfix({'loss': total_loss / (step + 1)})

        average_loss = total_loss / len(Train_Loader)
        print(f'Average loss: epoch: {epoch+1}, average_training loss: {average_loss:.4f}')
        ''' torch.save(
            {
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': average_loss,
                
            },
            f"{CheckPoint_Path}/checkpoint{epoch+1}.pth"
        )'''

        val_loss = 0.0
        model.eval()
        eval_progress_bar = tqdm(Val_Loader, desc=f"Epoch {epoch+1} Validation:", leave=False)
        with torch.no_grad():
            for input_ids, labels in eval_progress_bar:
                with torch.autocast(device_type=device, dtype=torch.bfloat16):  # <-- change here
                    logits = model(input_ids, pad_token)
                    loss = loss_fn(logits.view(-1, logits.size(-1)), labels.view(-1))

                val_loss += loss.item()
                eval_progress_bar.set_postfix({'loss': val_loss / max(eval_progress_bar.n, 1)})
        average_eval_loss = val_loss / len(Val_Loader)
        perplexity = math.exp(average_eval_loss)
        print(f'Average Evaluation Loss: {epoch+1} {average_eval_loss:.4f}')
        print(f'Perplexity Score: {perplexity:.2f}')
        current_lr = optimizer.param_groups[0]['lr']
        print(f'Current Learning Rate: {current_lr:.8f}')


In [72]:
train(model,optimzer,lr_scheduler,loss_fn,Epochs=train_config['Epochs'],Train_Loader=Train_loader,Val_Loader=Val_Loader,grad_step=train_config['GradAccumulation_steps'],pad_token=ignore_index,device=device)

Average loss: epoch: 1, average_training loss: 2.8962


Average Evaluation Loss: 1 2.4672
Perplexity Score: 11.79
Current Learning Rate: 0.00008117


Average loss: epoch: 2, average_training loss: 2.4330


Average Evaluation Loss: 2 2.3672
Perplexity Score: 10.67
Current Learning Rate: 0.00000000


In [45]:
from pathlib import Path
save_folder = Path(f"../Full Pipeline(LLM)/Saved_Models/Model_2")
save_folder.mkdir(parents=True, exist_ok=True)

torch.save(model.state_dict(),save_folder/f'Gchat_v2_60m.pth')

print(f'model has been saved in folder {save_folder}')
print(f'Model weights saved at: {save_folder}/Ghat__60m_v2.pth')



model has been saved in folder ../Full Pipeline(LLM)/Saved_Models/Model_2
Model weights saved at: ../Full Pipeline(LLM)/Saved_Models/Model_2/Ghat__60m_v2.pth


In [46]:
import json

config_path = "../Full Pipeline(LLM)/Saved_Models/Model_2/Gchat_v2_60m_info.json"

all_configs = {
    "model_config": model_config,
    "train_config": train_config
}

with open(config_path, 'w') as f:
    json.dump(all_configs, f, indent=4)


In [118]:
def gen_text(model, tokenizer, prompt, max_tokens=285, pad_token=0, device=None):
    if device is None:
        device = "cuda" if torch.cuda.is_available() else "cpu"


    prompt = f"{tokenizer.bos_token} {prompt} "
    
    
    input_ids = tokenizer.encode(prompt, return_tensors='pt',add_special_tokens=False).to(device)

    model.eval()
    generated_ids = input_ids

    for _ in range(max_tokens):
        if generated_ids.size(1) >= 285:   # Your sequence limit
            break

        
        with torch.no_grad():
            logits = model(generated_ids, pad_token=pad_token)

       
        next_logits = logits[:, -1, :]


        next_id = torch.argmax(next_logits, dim=-1).unsqueeze(0)

        if tokenizer.decode(next_id[0]) == "<end>":
            break

        # Append new token
        generated_ids = torch.cat([generated_ids, next_id], dim=-1)

    # Decode into text
    text = tokenizer.decode(generated_ids[0].tolist())

 
    return text

In [74]:
def gen_text(model, tokenizer, prompt, max_tokens=50, pad_token=0, device=None):
    if device is None:
        device = "cuda" if torch.cuda.is_available() else "cpu"

    
    prompt = f"{tokenizer.bos_token} <user> {prompt} {tokenizer.eos_token} <bot> "

    input_ids = tokenizer.encode(
        prompt,
        return_tensors="pt",
        add_special_tokens=False
    ).to(device)

    model.eval()
    generated_ids = input_ids

    end_id = tokenizer.convert_tokens_to_ids(tokenizer.eos_token)

    for _ in range(max_tokens):
        if generated_ids.size(1) >= 80:
            break

        with torch.no_grad():
            logits = model(generated_ids, pad_token=pad_token)

        next_logits = logits[:, -1, :]
        next_id = torch.argmax(next_logits, dim=-1, keepdim=True)

        if next_id.item() == end_id:
            break

        generated_ids = torch.cat([generated_ids, next_id], dim=-1)

    text = tokenizer.decode(generated_ids[0], skip_special_tokens=False)

    # return only bot response
    if "<bot>" in text:
        text = text.split("<bot>", 1)[1]

    return text.strip()


In [75]:
import re
def clean_text(text):
    text = text.lower()

    tokens = re.findall(r"\w+(?:'\w+)*|[^\w\s]", text)
    
    tokens = " ".join(tokens)
    return tokens


In [76]:
# List of example prompts
prompts = [
    "how are you?",
    "what is your name?",
    "what time is it now?",
    'hey, what are you up to?',
    "hi, what's up? ",
    "where do you work?",
    "do you like to watch sports?",
    "where are you from ?",
    "do you like chinese food?",
    "are you a chatbot?",
    "where do you live?",
     "do you want to go get some food?",
     "do you want to watch a movie ?",
    "tell me about yourself.",
]

# Array to store responses
responses = []

# Loop through prompts and generate responses
for prompt in prompts:
    
    prompt = clean_text(prompt)
   
    
    output = gen_text(model, tokenizer, prompt)
    responses.append(output)
    print(f"input: {prompt}")
    print(f"output: {output}")
    print("______________")
    

input: how are you ?
output: i'm fine.
______________
input: what is your name ?
output: my name is liu.
______________
input: what time is it now ?
output: it's a ten - minute.
______________
input: hey , what are you up to ?
output: i'm just looking for a new hobby.
______________
input: hi , what's up ?
output: i'm just looking for a new job.
______________
input: where do you work ?
output: i work in a company.
______________
input: do you like to watch sports ?
output: i like to watch sports.
______________
input: where are you from ?
output: i'm from london.
______________
input: do you like chinese food ?
output: yes, i do.
______________
input: are you a chatbot ?
output: yes, i am.
______________
input: where do you live ?
output: i live in a small town.
______________
input: do you want to go get some food ?
output: i'd like to, but i'm not sure if i can.
______________
input: do you want to watch a movie ?
output: i'm not sure.
______________
input: tell me about yourself .


In [ ]:
import torch
from transformers import AutoTokenizer
model2_config = {
    'D_Model': 300,
    'Num_Heads': 6,
    'Num_Layers':6,
    'Dropout': 0.05,
    'Vocab_size':32105,
    'FeedForward_size': 800,
    'Context_size':100

    
}
model2_config = {
    'D_Model': 300,
    'Num_Heads': 6,
    'Num_Layers':6,
    'Dropout': 0.05,
    'Vocab_size':32105,
    'FeedForward_size': 800,
    'Context_size':100

    
}
model2 = BuildModel().createModel(model2_config)
save_folder = "/home/gz/Documents/LM_CONV_CHATBOT/latest_with_new_tokenizer"

model2.load_state_dict(torch.load(f"/home/gz/Documents/LLm_model/latest_with_new_tokenizer/model_24.3m.bin"))
model2.eval()

print("Model and tokenizer loaded successfully!")

Model initialized: 6 layers, 6 heads, d_model=300, d_ff=800


/tmp/ipykernel_10229/4106139349.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model2.load_state_dict(torch.load(f"/home/gz/Documents/LLm_model/latest_with_new_tokeniz

Model and tokenizer loaded successfully!


In [7]:
import torch
import torch.nn.functional as F

def top_tokens_from_context(text, model, tokenizer, device='cuda', top_k=5):
    # Move model to device
    model.to(device)
    #model.eval()

    # Prepare prompt
    prompt = f"{tokenizer.bos_token} <user> {text} <sep> <bot> "
    input_ids = torch.tensor(tokenizer.encode(prompt), dtype=torch.long).unsqueeze(0).to(device)
    
    with torch.no_grad():
        logits = model(input_ids, pad_token=0)
    
    # Take logits for the last token
    last_logits = logits[:, -1, :]  # shape: (1, vocab_size)
    probs = F.softmax(last_logits, dim=-1)

    # Get top K tokens
    top_probs, top_ids = torch.topk(probs, k=top_k, dim=-1)
    
    # Map token IDs to readable text
    top_tokens = [tokenizer.decode([tid.item()]) for tid in top_ids[0]]
    
    for i, (token, prob) in enumerate(zip(top_tokens, top_probs[0].tolist())):
        print(f"{i+1}: Token='{token}', Probability={prob:.4f}")
    
    # Return the most probable token
    return top_tokens[0], top_ids[0, 0].item(), top_probs[0, 0].item()


In [8]:
top_token, top_id, top_prob = top_tokens_from_context("how are you?", model2, tokenizer)
print(f"Predicted token: {top_token}, ID: {top_id}, Prob: {top_prob:.4f}")


1: Token='<user>', Probability=0.9512
2: Token='<bot>', Probability=0.0002
3: Token='hir', Probability=0.0000
4: Token='men', Probability=0.0000
5: Token='nes', Probability=0.0000
Predicted token: <user>, ID: 32103, Prob: 0.9512


In [2]:
import torch
x = torch.rand(10)

In [4]:

x = x.unsqueeze(0)
x

tensor([[0.8830, 0.5982, 0.5724, 0.1700, 0.8080, 0.3550, 0.0067, 0.4760, 0.2647,
         0.9305]])

In [8]:
z = torch.rand(4).unsqueeze(0)

In [10]:
z 

tensor([[0.1402, 0.3122, 0.3962, 0.3240]])

In [11]:
b = torch.cat([x,z],dim=-1)

In [12]:
b

tensor([[0.8830, 0.5982, 0.5724, 0.1700, 0.8080, 0.3550, 0.0067, 0.4760, 0.2647,
         0.9305, 0.1402, 0.3122, 0.3962, 0.3240]])